In [1]:
# Loading up all the necessary libraries. 

import qgrid
qgrid.set_grid_option('forceFitColumns', False)
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [47]:
# Ingesting the raw data
data = pd.read_csv('/Users/enochosei/Library/Mobile Documents/com~apple~CloudDocs/Data Science/E-Commerce UK/data.csv', encoding = 'ISO-8859-1')

In [48]:
data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
519935,580188,22907,PACK OF 20 NAPKINS PANTRY DESIGN,6,12/2/2011 12:02,0.85,16440.0,United Kingdom
50843,540562,20728,LUNCH BAG CARS BLUE,10,1/10/2011 10:35,1.65,12524.0,Germany
486112,577700,21616,4 PEAR BOTANICAL DINNER CANDLES,3,11/21/2011 12:06,1.25,14449.0,United Kingdom
388298,570423,22736,RIBBON REEL MAKING SNOWMEN,5,10/10/2011 13:38,1.65,14451.0,United Kingdom
17227,537691,22666,RECIPE BOX PANTRY YELLOW DESIGN,6,12/8/2010 10:39,2.95,13842.0,United Kingdom
316605,564755,22189,CREAM HEART CARD HOLDER,1,8/30/2011 10:31,8.29,NaN,United Kingdom
496903,578347,22944,CHRISTMAS METAL POSTCARD WITH BELLS,18,11/24/2011 9:26,0.97,NaN,United Kingdom
375621,569469,23010,CIRCUS PARADE BABY GIFT SET,2,10/4/2011 12:22,16.95,16360.0,United Kingdom
328076,565751,23241,TREASURE TIN GYMKHANA DESIGN,6,9/6/2011 12:56,2.08,16393.0,United Kingdom
155935,550034,20728,LUNCH BAG CARS BLUE,10,4/14/2011 11:48,1.65,16187.0,United Kingdom


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [50]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


## Data Cleaning

#### Dealing with the duplicates

In [51]:
duplicates = data.duplicated()
total_duplicated_rows = duplicates.sum()
total_duplicated_rows

5268

In [52]:
data_no_dupes = data.drop_duplicates(keep='first')
data_no_dupes.shape

(536641, 8)

#### Handling missing values

In [53]:
# rid the original dataframe of duplicates by overwriting it. 
data = data_no_dupes 

In [54]:
missing_values = data.isnull().sum()
missing_values

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135037
Country             0
dtype: int64

In [55]:
# Handling the missing values in 'Description' and 'CustomierID'.

data['Description'] = data['Description'].fillna('Unknown') # set purchases with no item description as unknown.
data['CustomerID'] = data['CustomerID'].fillna(0) # set purchases with no ID as unknown_customers with ID of 0. 

In [56]:
missing_values = data.isnull().sum()
missing_values

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

#### Ensure *data types* are of the correct format. 

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536641 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    536641 non-null  object 
 1   StockCode    536641 non-null  object 
 2   Description  536641 non-null  object 
 3   Quantity     536641 non-null  int64  
 4   InvoiceDate  536641 non-null  object 
 5   UnitPrice    536641 non-null  float64
 6   CustomerID   536641 non-null  float64
 7   Country      536641 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.8+ MB


In [58]:
data.sample(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
228903,556965,21915,RED HARMONICA IN BOX,60,6/16/2011 9:49,1.25,15129.0,United Kingdom


In [59]:
# converting data types to appropriate. 
data['Quantity'] = pd.to_numeric(data['Quantity'])
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [60]:
data['InvoiceNo'] = pd.to_numeric(data['InvoiceNo'])

ValueError: Unable to parse string "C536379" at position 141

In [69]:
# Investigating the error location
data.iloc[138:145]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
139,536381,22719,GUMBALL MONOCHROME COAT RACK,36,2010-12-01 09:41:00,1.06,15311.0,United Kingdom
140,536381,21523,DOORMAT FANCY FONT HOME SWEET HOME,10,2010-12-01 09:41:00,6.75,15311.0,United Kingdom
142,536382,10002,INFLATABLE POLITICAL GLOBE,12,2010-12-01 09:45:00,0.85,16098.0,United Kingdom
143,536382,21912,VINTAGE SNAKES LADDERS,8,2010-12-01 09:45:00,3.75,16098.0,United Kingdom
144,536382,21832,CHOCOLATE CALCULATOR,12,2010-12-01 09:45:00,1.65,16098.0,United Kingdom
145,536382,22411,JUMBO SHOPPER VINTAGE RED PAISLEY,10,2010-12-01 09:45:00,1.95,16098.0,United Kingdom
146,536382,22379,RECYCLING BAG RETROSPOT,10,2010-12-01 09:45:00,2.10,16098.0,United Kingdom


 We have three issues here:
 - The Invoice number is not a number.
 - The Stock code is a letter.
 - The Quanity Value is Negative.
 
The description is 'Discount', indicating something intersting happening here.

To deal we will deal with this later. 

#### Engineering Relevant Columns

In [70]:
# A new column for the delineated datetime elements. 

data['Time'] = data['InvoiceDate'].dt.time
data['Day'] = data['InvoiceDate'].dt.day_name()
data['Month'] = data['InvoiceDate'].dt.month_name()
data['Year'] = data['InvoiceDate'].dt.year

# A new column for the _revenue_ of a sale. 
data['Revenue'] = data['Quantity']*data['UnitPrice']

In [71]:
data.sample(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Time,Day,Month,Year,Revenue
350942,567656,21035,SET/2 RED RETROSPOT TEA TOWELS,2,2011-09-21 14:40:00,6.63,14096.0,United Kingdom,14:40:00,Wednesday,September,2011,13.26
94115,544329,21915,RED HARMONICA IN BOX,12,2011-02-17 16:43:00,1.25,17288.0,United Kingdom,16:43:00,Thursday,February,2011,15.00
15008,537629,21889,WOODEN BOX OF DOMINOES,12,2010-12-07 15:03:00,1.25,17419.0,United Kingdom,15:03:00,Tuesday,December,2010,15.00


#### Dealing with Data Issues;  - The Quanity Value is Negative.

In [72]:
neg_quant_data = data[data['Quantity']<0]
neg_quant_data.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Time,Day,Month,Year,Revenue
87404,543654,22525,Unknown,-57,2011-02-11 10:29:00,0.00,0.0,United Kingdom,10:29:00,Friday,February,2011,-0.00
280720,561486,21765,Unknown,-2,2011-07-27 13:48:00,0.00,0.0,United Kingdom,13:48:00,Wednesday,July,2011,-0.00
18134,C537797,22469,HEART OF WICKER SMALL,-1,2010-12-08 13:07:00,1.45,13113.0,United Kingdom,13:07:00,Wednesday,December,2010,-1.45
159708,C550353,22654,DELUXE SEWING KIT,-1,2011-04-18 10:29:00,5.95,17841.0,United Kingdom,10:29:00,Monday,April,2011,-5.95
170530,C551316,21703,BAG 125g SWIRLY MARBLES,-12,2011-04-27 15:12:00,0.42,15220.0,United Kingdom,15:12:00,Wednesday,April,2011,-5.04


In [73]:
neg_quant_data.describe()

,Quantity,UnitPrice,CustomerID,Year,Revenue
count,10006.000000,10006.000000,10006.000000,10006.000000,10006.000000
mean,-47.668999,3.832068,12748.554967,2010.924146,-47.568075
std,1125.195180,8.644141,5578.074388,0.264778,1857.128573
min,-80995.000000,0.000000,0.000000,2010.000000,-168469.600000
25%,-12.000000,0.850000,12753.000000,2011.000000,-15.000000
50%,-2.000000,2.080000,14426.000000,2011.000000,-6.240000
75%,-1.000000,4.250000,16032.000000,2011.000000,-1.950000
max,-1.000000,295.000000,18282.000000,2011.000000,-0.000000


In [74]:
neg_quant_data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Time,Day,Month,Year,Revenue
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,09:49:00,Wednesday,December,2010,-4.65
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,10:24:00,Wednesday,December,2010,-19.80
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,10:24:00,Wednesday,December,2010,-6.96
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,10:24:00,Wednesday,December,2010,-6.96
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,10:24:00,Wednesday,December,2010,-6.96


In [75]:
# Drop negative quantity row. Unecessary for the Anlysis of genuine sales transactions. 
data = data.drop(neg_quant_data.index)

In [76]:
# Free items
free_items = data[data['UnitPrice']== 0]
qgrid.show_grid(free_items)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [77]:
#Finding the unique names of the item descriptions of the free items. 
free_items['Description'].unique()

array(['Unknown', 'amazon', 'ROUND CAKE TIN VINTAGE GREEN',
       'CREAM SWEETHEART LETTER RACK', 'ZINC WILLIE WINKIE  CANDLE STICK',
       'BOX OF 24 COCKTAIL PARASOLS', 'DOORMAT ENGLISH ROSE ',
       'DOORMAT 3 SMILEY CATS', 'GREEN REGENCY TEACUP AND SAUCER',
       'FRENCH BLUE METAL DOOR SIGN 7', 'FRENCH BLUE METAL DOOR SIGN 5',
       'FRENCH BLUE METAL DOOR SIGN 6', 'FRENCH BLUE METAL DOOR SIGN 4',
       'FRENCH BLUE METAL DOOR SIGN No', 'FRENCH BLUE METAL DOOR SIGN 8',
       'FRENCH BLUE METAL DOOR SIGN 1', 'RED KITCHEN SCALES',
       'IVORY KITCHEN SCALES', 'SET OF 6 SOLDIER SKITTLES',
       'CHILDS GARDEN TROWEL BLUE ', 'CHILDRENS GARDEN GLOVES BLUE',
       'PICNIC BASKET WICKER SMALL', 'PICNIC BASKET WICKER LARGE',
       'EMPIRE UNION JACK TV DINNER TRAY',
       'TV DINNER TRAY VINTAGE PAISLEY', 'SPACEBOY TV DINNER TRAY',
       'TV DINNER TRAY DOLLY GIRL', 'CHILDS GARDEN SPADE BLUE',
       'CHILDS GARDEN RAKE BLUE', 'WATERING CAN PINK BUNNY',
       'ENAMEL FIRE B

In [78]:
free_items_good_descr = free_items[free_items['Description'].str.isupper()]

In [79]:
free_items_good_descr.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Time,Day,Month,Year,Revenue
9302,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647.0,Germany,14:02:00,Sunday,December,2010,0.0
14335,537534,85064,CREAM SWEETHEART LETTER RACK,1,2010-12-07 11:48:00,0.0,0.0,United Kingdom,11:48:00,Tuesday,December,2010,0.0
14336,537534,84832,ZINC WILLIE WINKIE CANDLE STICK,1,2010-12-07 11:48:00,0.0,0.0,United Kingdom,11:48:00,Tuesday,December,2010,0.0
14337,537534,84692,BOX OF 24 COCKTAIL PARASOLS,2,2010-12-07 11:48:00,0.0,0.0,United Kingdom,11:48:00,Tuesday,December,2010,0.0
14338,537534,48184,DOORMAT ENGLISH ROSE,3,2010-12-07 11:48:00,0.0,0.0,United Kingdom,11:48:00,Tuesday,December,2010,0.0


In [80]:
free_items_good_descr['Description'].unique()

array(['ROUND CAKE TIN VINTAGE GREEN', 'CREAM SWEETHEART LETTER RACK',
       'ZINC WILLIE WINKIE  CANDLE STICK', 'BOX OF 24 COCKTAIL PARASOLS',
       'DOORMAT ENGLISH ROSE ', 'DOORMAT 3 SMILEY CATS',
       'GREEN REGENCY TEACUP AND SAUCER', 'FRENCH BLUE METAL DOOR SIGN 7',
       'FRENCH BLUE METAL DOOR SIGN 5', 'FRENCH BLUE METAL DOOR SIGN 6',
       'FRENCH BLUE METAL DOOR SIGN 4', 'FRENCH BLUE METAL DOOR SIGN 8',
       'FRENCH BLUE METAL DOOR SIGN 1', 'RED KITCHEN SCALES',
       'IVORY KITCHEN SCALES', 'SET OF 6 SOLDIER SKITTLES',
       'CHILDS GARDEN TROWEL BLUE ', 'CHILDRENS GARDEN GLOVES BLUE',
       'PICNIC BASKET WICKER SMALL', 'PICNIC BASKET WICKER LARGE',
       'EMPIRE UNION JACK TV DINNER TRAY',
       'TV DINNER TRAY VINTAGE PAISLEY', 'SPACEBOY TV DINNER TRAY',
       'TV DINNER TRAY DOLLY GIRL', 'CHILDS GARDEN SPADE BLUE',
       'CHILDS GARDEN RAKE BLUE', 'WATERING CAN PINK BUNNY',
       'ENAMEL FIRE BUCKET CREAM', 'ENAMEL FLOWER JUG CREAM',
       'AIRLINE BAG V

In [81]:
bad_descr_free_items = free_items.drop(free_items_good_descr.index)
bad_descr_free_items.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Time,Day,Month,Year,Revenue
511451,579500,22810,Unknown,33,2011-11-29 16:14:00,0.0,0.0,United Kingdom,16:14:00,Tuesday,November,2011,0.0
431590,573664,21192,wrongly marked,48,2011-10-31 16:07:00,0.0,0.0,United Kingdom,16:07:00,Monday,October,2011,0.0
336412,566325,22035,taig adjust,36,2011-09-12 12:03:00,0.0,0.0,United Kingdom,12:03:00,Monday,September,2011,0.0
20037,538004,22114,Unknown,144,2010-12-09 12:11:00,0.0,0.0,United Kingdom,12:11:00,Thursday,December,2010,0.0
299544,563105,22194,Unknown,2,2011-08-12 10:15:00,0.0,0.0,United Kingdom,10:15:00,Friday,August,2011,0.0


In [82]:
bad_descr_free_items.shape[1]

13

In [83]:
#Drop bad free items from the data.
data = data.drop(bad_descr_free_items.index)

In [84]:
# All correct item descriptions are all caps. We find the ones with incorrect labelling. 

not_all_caps = data[~data['Description'].str.isupper()]
unique_not_all_caps = not_all_caps['Description'].unique()

print(unique_not_all_caps)

['BAG 500g SWIRLY MARBLES' 'POLYESTER FILLER PAD 45x45cm'
 'BAG 125g SWIRLY MARBLES' 'BAG 250g SWIRLY MARBLES'
 'POLYESTER FILLER PAD 45x30cm' 'POLYESTER FILLER PAD 40x40cm'
 'FRENCH BLUE METAL DOOR SIGN No' 'Dr. Jams Arouzer Stress Ball'
 '3 TRADITIONAl BISCUIT CUTTERS  SET' 'FOLK ART GREETING CARDpack/12'
 'ESSENTIAL BALM 3.5g TIN IN ENVELOPE' 'POLYESTER FILLER PAD 65CMx65CM'
 'Dads Cab Electronic Meter' 'NUMBER TILE VINTAGE FONT No '
 'Boombox Ipod Classic' 'NUMBER TILE COTTAGE GARDEN No'
 'USB Office Mirror Ball' 'POLYESTER FILLER PAD 30CMx30CM'
 'Dotcomgiftshop Gift Voucher 40.00' 'Dotcomgiftshop Gift Voucher 50.00'
 'Dotcomgiftshop Gift Voucher 30.00' 'Dotcomgiftshop Gift Voucher 20.00'
 'Dotcomgiftshop Gift Voucher 10.00' 'POLYESTER FILLER PAD 60x40cm'
 'FLOWERS HANDBAG blue and orange' 'Next Day Carriage'
 'THE KING GIFT BAG 25x24x12cm' 'Dotcomgiftshop Gift Voucher 100.00'
 'High Resolution Image']


In [85]:
not_all_caps

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Time,Day,Month,Year,Revenue
482,536408,21705,BAG 500g SWIRLY MARBLES,12,2010-12-01 11:41:00,1.65,14307.0,United Kingdom,11:41:00,Wednesday,December,2010,19.80
918,536500,46000M,POLYESTER FILLER PAD 45x45cm,10,2010-12-01 12:35:00,1.55,17377.0,United Kingdom,12:35:00,Wednesday,December,2010,15.50
1961,536544,21703,BAG 125g SWIRLY MARBLES,1,2010-12-01 14:32:00,0.85,0.0,United Kingdom,14:32:00,Wednesday,December,2010,0.85
1962,536544,21704,BAG 250g SWIRLY MARBLES,1,2010-12-01 14:32:00,1.66,0.0,United Kingdom,14:32:00,Wednesday,December,2010,1.66
2038,536557,46000R,POLYESTER FILLER PAD 45x30cm,2,2010-12-01 14:41:00,1.45,17841.0,United Kingdom,14:41:00,Wednesday,December,2010,2.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540653,581492,21703,BAG 125g SWIRLY MARBLES,1,2011-12-09 10:03:00,0.83,0.0,United Kingdom,10:03:00,Friday,December,2011,0.83
540654,581492,21704,BAG 250g SWIRLY MARBLES,1,2011-12-09 10:03:00,1.63,0.0,United Kingdom,10:03:00,Friday,December,2011,1.63
541054,581492,22965,3 TRADITIONAl BISCUIT CUTTERS SET,1,2011-12-09 10:03:00,4.13,0.0,United Kingdom,10:03:00,Friday,December,2011,4.13
541612,581514,21705,BAG 500g SWIRLY MARBLES,84,2011-12-09 11:20:00,0.39,17754.0,United Kingdom,11:20:00,Friday,December,2011,32.76


In [86]:
data[data['Description'].str.contains('debt',case = False,na='False')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Time,Day,Month,Year,Revenue


In [87]:
#Valid Stock codes have some form of numerical element. We identify transactions with invalid stockcodes. 
incorrect_stock_code = data[~data['StockCode'].str.contains('\d', regex=True)]
print (incorrect_stock_code['StockCode'].unique(),
incorrect_stock_code['Description'].unique())

[] []


In [88]:
incorrect_stock_code

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Time,Day,Month,Year,Revenue


In [89]:
# Remove Invalid Stock Code Items
data = data.drop(incorrect_stock_code.index)

In [90]:
# All correct item descriptions are all caps. We find the ones with incorrect labelling. 

not_all_caps = data[~data['Description'].str.isupper()]
unique_not_all_caps = not_all_caps['Description'].unique()

print(unique_not_all_caps)

['BAG 500g SWIRLY MARBLES' 'POLYESTER FILLER PAD 45x45cm'
 'BAG 125g SWIRLY MARBLES' 'BAG 250g SWIRLY MARBLES'
 'POLYESTER FILLER PAD 45x30cm' 'POLYESTER FILLER PAD 40x40cm'
 'FRENCH BLUE METAL DOOR SIGN No' 'Dr. Jams Arouzer Stress Ball'
 '3 TRADITIONAl BISCUIT CUTTERS  SET' 'FOLK ART GREETING CARDpack/12'
 'ESSENTIAL BALM 3.5g TIN IN ENVELOPE' 'POLYESTER FILLER PAD 65CMx65CM'
 'Dads Cab Electronic Meter' 'NUMBER TILE VINTAGE FONT No '
 'Boombox Ipod Classic' 'NUMBER TILE COTTAGE GARDEN No'
 'USB Office Mirror Ball' 'POLYESTER FILLER PAD 30CMx30CM'
 'Dotcomgiftshop Gift Voucher 40.00' 'Dotcomgiftshop Gift Voucher 50.00'
 'Dotcomgiftshop Gift Voucher 30.00' 'Dotcomgiftshop Gift Voucher 20.00'
 'Dotcomgiftshop Gift Voucher 10.00' 'POLYESTER FILLER PAD 60x40cm'
 'FLOWERS HANDBAG blue and orange' 'Next Day Carriage'
 'THE KING GIFT BAG 25x24x12cm' 'Dotcomgiftshop Gift Voucher 100.00'
 'High Resolution Image']


In [91]:
data['Description'] = data['Description'].str.replace('[^a-zA-Z0-9.xX/ ]','',regex=True)

In [92]:
not_all_caps = data[~data['Description'].str.isupper()]
unique_not_all_caps = not_all_caps['Description'].unique()

print(unique_not_all_caps)

['BAG 500g SWIRLY MARBLES' 'POLYESTER FILLER PAD 45x45cm'
 'BAG 125g SWIRLY MARBLES' 'BAG 250g SWIRLY MARBLES'
 'POLYESTER FILLER PAD 45x30cm' 'POLYESTER FILLER PAD 40x40cm'
 'FRENCH BLUE METAL DOOR SIGN No' 'Dr. Jams Arouzer Stress Ball'
 '3 TRADITIONAl BISCUIT CUTTERS  SET' 'FOLK ART GREETING CARDpack/12'
 'ESSENTIAL BALM 3.5g TIN IN ENVELOPE' 'POLYESTER FILLER PAD 65CMx65CM'
 'Dads Cab Electronic Meter' 'NUMBER TILE VINTAGE FONT No '
 'Boombox Ipod Classic' 'NUMBER TILE COTTAGE GARDEN No'
 'USB Office Mirror Ball' 'POLYESTER FILLER PAD 30CMx30CM'
 'Dotcomgiftshop Gift Voucher 40.00' 'Dotcomgiftshop Gift Voucher 50.00'
 'Dotcomgiftshop Gift Voucher 30.00' 'Dotcomgiftshop Gift Voucher 20.00'
 'Dotcomgiftshop Gift Voucher 10.00' 'POLYESTER FILLER PAD 60x40cm'
 'FLOWERS HANDBAG blue and orange' 'Next Day Carriage'
 'THE KING GIFT BAG 25x24x12cm' 'Dotcomgiftshop Gift Voucher 100.00'
 'High Resolution Image']


In [ ]:
# Replace the pound sign